# Accessing Databases via Web APIs: Lecture Code
* * * * *

In [1]:
# Import required libraries
import requests
import urllib
import json
from __future__ import division
import math

## 1. Constructing API GET Request

In the first place, we know that every call will require us to provide a) a base URL for the API, b) some authorization code or key, and c) a format for the response. So let's put store those in some variables.

In [37]:
# set key
key="c98c46b9278b4f4e98acb812ec3d9dbb"

# set base url
base_url="http://api.nytimes.com/svc/search/v2/articlesearch"

# set response format
response_format=".json"

You often want to send some sort of data in the URL’s query string. This data tells the API what information you want. In our case, we want articles about Prince. Requests allows you to provide these arguments as a dictionary, using the `params` keyword argument. In addition to the search term `q`, we have to put in the `api-key` term.

In [3]:
# set search parameters
search_params = {"q":"Prince",
                 "api-key":key}       

Now we're ready to make the request. We use the `.get` method from the `requests` library to make an HTTP GET Request.

In [4]:
# make request
r = requests.get(base_url+response_format, params=search_params)

Now, we have a [response](http://docs.python-requests.org/en/latest/api/#requests.Response) object called `r`. We can get all the information we need from this object. For instance, we can see that the URL has been correctly encoded by printing the URL. Click on the link to see what happens.

In [5]:
print(r.url)

http://api.nytimes.com/svc/search/v2/articlesearch.json?q=Prince&api-key=13ff759582b04c6d8050e178b2dc8d0e


Click on that link to see what it returns!

Hmm - looks like we're getting a lot of junk unrelated to the musical artist in here. Let's make it a little more specific:

In [6]:
# set search parameters
search_params = {"q":"Prince music",
                 "api-key":key}  
# make request
r = requests.get(base_url+response_format, params=search_params)

print(r.url)

http://api.nytimes.com/svc/search/v2/articlesearch.json?q=Prince+music&api-key=13ff759582b04c6d8050e178b2dc8d0e


Better!

### Challenge 1:  Adding a date range

What if we only want to search within a particular date range? The NYT Article API allows us to specify start and end dates.

Alter the `search_params` code above so that the request only searches for articles in the year 2005.

You're gonna need to look at the documentation [here](http://developer.nytimes.com/docs/read/article_search_api_v2) to see how to do this.

In [19]:
#YOUR CODE HERE

# Uncomment to test
# r = requests.get(base_url+response_format, params=search_params)
# r.url

'http://api.nytimes.com/svc/search/v2/articlesearch.json?q=Prince+music&api-key=13ff759582b04c6d8050e178b2dc8d0e'

### Challenge 2:  Specifying a results page

The above will return the first 10 results. To get the next ten, you need to add a "page" parameter. Change the search parameters above to get the second 10 resuls. 

In [38]:
search_params = {"q":"Prince music",
                 "begin_date": "20050101",
                 "end_date": "20050131",
                 "page": "1",
                 "api-key":key}  
# make request
r = requests.get(base_url+response_format, params=search_params)

print(r.url)

# Uncomment to test
# r = requests.get(base_url+response_format, params=search_params)
# r.url

http://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=c98c46b9278b4f4e98acb812ec3d9dbb&q=Prince+music&end_date=20050131&begin_date=20050101&page=1


## 2. Parsing the response text

We can read the content of the server’s response using `.text`

In [9]:
# Inspect the content of the response, parsing the result as text
response_text= r.text
print(response_text[:1000])

{"response":{"meta":{"hits":19,"time":86,"offset":10},"docs":[{"web_url":"http:\/\/www.nytimes.com\/2005\/01\/13\/arts\/dance\/13cind.html","snippet":"The American premiere of Aleksei Ratmansky's version of \"Cinderella\" was a wildly uneven occasion, choreographically eccentric and scenically impoverished.","lead_paragraph":"The American premiere of Aleksei Ratmansky's version of ''Cinderella'' for the Kirov Ballet in the Kennedy Center Opera House on Monday night was a wildly uneven occasion, choreographically eccentric and scenically impoverished. The impoverishment was due in part to Mr. Ratmansky's incoherently chic conception, in part to the inevitable tweakings and trimmings this production must have gone through over its three years of life and to the exigencies of the steady touring to which most Kirov productions are subjected.","abstract":"John Rockwell reviews American premiere of Aleksei Ramansky's version of Cinderella for Kirov Ballet at Kennedy Center Opera House; photo

What you see here is JSON text, encoded as unicode text. JSON stands for "Javascript object notation." It has a very similar structure to a python dictionary -- both are built on key/value pairs. This makes it easy to convert JSON response to a python dictionary.

In [10]:
# Convert JSON response to a dictionary
data=json.loads(response_text)
# data

That looks intimidating! But it's really just a big dictionary. Let's see what keys we got in there.

In [11]:
data.keys()

dict_keys(['response', 'copyright', 'status'])

In [12]:
# this is boring
data['status']

'OK'

In [13]:
# so is this
data['copyright']

'Copyright (c) 2013 The New York Times Company.  All Rights Reserved.'

In [14]:
# this is what we want!
# data['response']

In [16]:
data['response'].keys()

dict_keys(['docs', 'meta'])

In [17]:
data['response']['meta'].keys()

dict_keys(['offset', 'time', 'hits'])

In [18]:
data['response']['meta']['hits'] # whoa - that's a lot of hits!

19

In [19]:
# data['response']['docs']
type(data['response']['docs'])

list

That looks what we want! Let's put that in it's own variable.

In [20]:
docs = data['response']['docs']

In [23]:
docs[1]

{'_id': '57f3fbc2253f0a30af7acbce',
 'abstract': 'Opening of newly completed Paisley Park museum, in former studio of musician Prince, is delayed as Chanhassen, Minn, City Council postpones rezoning request.',
 'blog': [],
 'byline': {'original': 'By BEN SISARIO',
  'person': [{'firstname': 'Ben',
    'lastname': 'SISARIO',
    'organization': '',
    'rank': 1,
    'role': 'reported'}]},
 'document_type': 'article',
 'headline': {'main': 'Prince’s Paisley Park Museum Opening Snared by Zoning Vote',
  'print_headline': 'Opening Challenge for Prince Museum'},
 'keywords': [{'is_major': 'N',
   'name': 'subject',
   'rank': '1',
   'value': 'Museums'},
  {'is_major': 'N',
   'name': 'persons',
   'rank': '3',
   'value': 'Prince (1958- )'},
  {'is_major': 'Y',
   'name': 'glocations',
   'rank': '4',
   'value': 'Chanhassen (Minn)'},
  {'is_major': 'Y', 'name': 'subject', 'rank': '5', 'value': 'Zoning'},
  {'is_major': 'N',
   'name': 'subject',
   'rank': '6',
   'value': 'Pop and Rock 

## 3. Putting everything together to get all the articles.

### That's great. But we only have 10 items. The original response said we had 30528 hits! Which means we have to make 30528 /10, or 3053 requests to get them all.*
#### *Note - in general, most free APIs have limits on how often you can "call" them, i.e., how many requests you can send. For the NYT, the daily limit is 1000, and you are limited to 5 calls/sec. At first, this might sound like plenty--but in general, it's not. You'll want to be creative with your search terms and date restrictions to ensure that you have a manageable number of calls to the API. Going forward, we're just going to look at the 2016 results - a much more manageable 523 (53 calls). 

### Sounds like a job for a loop! 

But first, let's review what we've done so far.

In [39]:
# set key
key="ef9055ba947dd842effe0ecf5e338af9:15:72340235"

# set base url
base_url="http://api.nytimes.com/svc/search/v2/articlesearch"

# set response format
response_format=".json"

# set search parameters
search_params = {"q":"Prince music",
                 "api-key":key,
                 "begin_date":"20150101", # date must be in YYYYMMDD format
                 "end_date":"20150131"}

# make request
rr = requests.get(base_url+response_format, params=search_params)
    
# convert to a dictionary
data=json.loads(rr.text)
    
# get number of hits
hits = data['response']['meta']['hits']
print("number of hits: " + str(hits))
    
# get number of pages
pages = int(math.ceil(hits/10))

number of hits: 52


In [40]:
# make an empty list where we'll hold all of our docs for every page
all_docs = [] 
    
# now we're ready to loop through the pages
for i in range(pages):
    print("collecting page " + str(i))
        
    # set the page parameter
    search_params['page'] = i
        
    # make request
    rr = requests.get(base_url+response_format, params=search_params)
    
    # get text and convert to a dictionary
    data=json.loads(rr.text)
        
    # get just the docs
    docs = data['response']['docs']
        
    # add those docs to the big list
    all_docs = all_docs + docs

collecting page 0
collecting page 1
collecting page 2


KeyError: 'response'

In [41]:
print(data)

{'message': 'API rate limit exceeded'}


In [31]:
len(all_docs)

10

### Challenge 3: Make a function

Turn the code above into a function that inputs a search term, and returns all the documents containing that search term in 2014.

In [47]:
def apisearch(q, begin):
    # set key
    key="c98c46b9278b4f4e98acb812ec3d9dbb"
    # set base url
    base_url="http://api.nytimes.com/svc/search/v2/articlesearch"
    # set response format
    response_format=".json" 
    search_params = {"q": q,
                 "begin_date": begin,
                 "end_date": "20141231",
                 "page": "1",
                 "api-key":key}  
    # make request
    r = requests.get(base_url+response_format, params=search_params)
    print(r.url)

In [48]:
apisearch("Clinton", "20140101")

http://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=c98c46b9278b4f4e98acb812ec3d9dbb&q=Clinton&end_date=20141231&begin_date=20140101&page=1


## 4. Formatting and Exporting

Let's take another look at one of these documents.

In [42]:
all_docs[0]

{'_id': '54b4353c38f0d8623a353df7',
 'abstract': 'Stephen Holden reviews performance celebrating theatrical collaborations of composer Stephen Sondheim and director Harold Prince as part of Lyrics and Lyricists series at 92nd Street Y.',
 'blog': [],
 'byline': {'contributor': '',
  'original': 'By STEPHEN HOLDEN',
  'person': [{'firstname': 'Stephen',
    'lastname': 'HOLDEN',
    'organization': '',
    'rank': 1,
    'role': 'reported'}]},
 'document_type': 'article',
 'headline': {'content_kicker': 'Music Review',
  'main': 'Celebrating 2 Broadway Powerhouses, Bit by Bit',
  'print_headline': 'Celebrating 2 Broadway Powerhouses, Bit by Bit',
  'seo': 'David Loud&#8217;s &#8216;A Good Thing Going&#8217; Honors Sondheim and Prince'},
 'keywords': [{'is_major': 'Y',
   'name': 'subject',
   'rank': '1',
   'value': 'Music'},
  {'is_major': 'Y',
   'name': 'organizations',
   'rank': '2',
   'value': '92nd Street Y'},
  {'is_major': 'Y',
   'name': 'persons',
   'rank': '3',
   'value'

This is all great, but it's pretty messy. What we’d really like to to have, eventually, is a CSV, with each row representing an article, and each column representing something about that article (header, date, etc). As we saw before, the best way to do this is to make a lsit of dictionaries, with each dictionary representing an article and each dictionary representing a field of metadata from that article (e.g. headline, date, etc.) We can do this with a custom function:

In [17]:
def format_articles(unformatted_docs):
    '''
    This function takes in a list of documents returned by the NYT api 
    and parses the documents into a list of dictionaries, 
    with 'id', 'header', and 'date' keys
    '''
    formatted = []
    for i in unformatted_docs:
        dic = {}
        dic['id'] = i['_id']
        dic['headline'] = i['headline']['main'].encode("utf8")
        dic['date'] = i['pub_date'][0:10] # cutting time of day.
        formatted.append(dic)
    return(formatted) 

In [18]:
all_formatted = format_articles(all_docs)

In [19]:
all_formatted[:5]

[{'date': '2016-09-17',
  'headline': b'Prince Memorial Concert to Feature Stevie Wonder and Others',
  'id': '57dc61e138f0d84ba6a0c6c6'},
 {'date': '2016-09-03',
  'headline': b'Prince\xe2\x80\x99s Old Band Resurrects Him Through His Songs',
  'id': '57c9f10f38f0d8656933ff9f'},
 {'date': '2016-08-26',
  'headline': b'Paisley Park, Prince\xe2\x80\x99s Estate, Will Open as a Museum',
  'id': '57bf145138f0d850d30c939e'},
 {'date': '2016-09-08',
  'headline': b'Another Side of Bob Dylan: Iron Sculptor',
  'id': '57d0545438f0d82f70ca102a'},
 {'date': '2016-07-09',
  'headline': b'The Revolution: A \xe2\x80\x98Purple Rain\xe2\x80\x99 Reunion in Prince\xe2\x80\x99s Honor',
  'id': '577fe62f38f0d8706f764119'}]

### Challenge 4: Export the data to a CSV.

In [24]:
import csv
keys = all_formatted[0].keys()
with open('article_API.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(all_formatted)
    